In [61]:
from sklearn import datasets, linear_model, preprocessing
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

camden_applications = 'Camden_Licensing_Applications_Beta.csv'
camden_business = 'Companies_Registered_In_Camden_And_Surrounding_Boroughs.csv'

df = pd.read_csv(camden_applications,parse_dates=['Received Date', 'Approved Date', 'Refused Date'])
#shuffle randomly
df = df.sample(frac=1).reset_index(drop=True)
df['year'] = df['Received Date'].dt.year

In [62]:
#iref = df.columns.get_loc('Refused Date')
#iapp = df.columns.get_loc('Approved Date')
#ircv = df.columns.get_loc('Received Date')

hasref = pd.notnull(df['Refused Date'])
hasapp = pd.notnull(df['Approved Date'])
hasrcv = pd.notnull(df['Received Date'])

#df = df[hasrcv & (hasapp | hasref)]
df = df[hasrcv & hasapp]

df['tduration'] = ((df['Approved Date'].dt.date -
                   df['Received Date'].dt.date))
df['duration'] = df['tduration'].dt.days
df = df[df['duration'] > 0]
data = df.as_matrix()

In [63]:
df

,Application Reference,Application Type,Premises Name,Premises Address,Application Legislation,Application Status,Application Objections,Approved Date,Date Of Events,Expiry Date,...,Longitude,Latitude,Spatial Accuracy,Last Uploaded,Location,Socrata ID,Organisation URI,year,tduration,duration
0,APP\PERSONAL-NEW\000039,Personal - New Licence,NaN,NaN,LA2003,Application Approved,No,2005-07-10,NaN,NaN,...,NaN,NaN,No Spatial Information,11/05/2017 07:36:46 PM,NaN,2370,http://opendatacommunities.org/id/london-borou...,2005.0,1 days,1
1,APP\PREMISES-NEW\000402,Premises - New Licence,Barrica,"62 GOODGE STREET, LONDON, W1T 4NE",LA2003,Application Approved In Full,Yes,2006-10-10,NaN,NaN,...,-0.137039,51.519350,Nearest OS Open Map-Local Building,11/05/2017 07:37:30 PM,"(51.51935, -0.137039)",29616,http://opendatacommunities.org/id/london-borou...,2006.0,56 days,56
2,APP\TEN\008939,Temporary Event Notice,Yeomanry House,"YEOMANRY HOUSE HANDEL STREET, LONDON, WC1N 1NP",LA2003,Application Approved,No,2013-01-17,NaN,NaN,...,-0.124169,51.525611,Nearest OS Open Map-Local Building,11/05/2017 07:37:22 PM,"(51.525611, -0.124169)",12677,http://opendatacommunities.org/id/london-borou...,2013.0,2 days,2
4,APP\TEN\010325,Temporary Event Notice,La Roma Bella,"62 GREAT RUSSELL STREET, LONDON, WC1B 3BG",LA2003,Application Approved,No,2014-09-05,NaN,NaN,...,-0.125312,51.518343,Nearest OS Open Map-Local Building,11/05/2017 07:37:22 PM,"(51.518343, -0.125312)",11428,http://opendatacommunities.org/id/london-borou...,2014.0,31 days,31
5,APP\PREMISES-TRANS\001322,Premises Licence - Transfer Application,"Paramount (Floors 31, 32, 33)",CENTRE POINT COMPLEX DEVELOPMENT SITE AT 101-1...,LA2003,Application Approved In Full,No,2015-01-29,NaN,NaN,...,-0.129265,51.516178,Nearest OS Open Map-Local Building,11/05/2017 07:37:29 PM,"(51.516178, -0.129265)",26977,http://opendatacommunities.org/id/london-borou...,2015.0,1 days,1
6,APP\TEN\010742,Temporary Event Notice,Senator International,"11 13 MELTON STREET, LONDON, NW1 2EA",LA2003,Application Approved,No,2014-03-10,NaN,NaN,...,-0.135339,51.527366,Nearest OS Open Map-Local Building,11/05/2017 07:37:21 PM,"(51.527366, -0.135339)",11070,http://opendatacommunities.org/id/london-borou...,2014.0,28 days,28
8,APP\TEN\010233,Temporary Event Notice,Parliament Hill Running Track,PARLIAMENT HILL FIELDS ATHLETICS TRACK HIGHGAT...,LA2003,Application Approved,No,2014-09-04,NaN,NaN,...,-0.156569,51.557186,Nearest OS Open Map-Local Building,11/05/2017 07:37:22 PM,"(51.557186, -0.156569)",11513,http://opendatacommunities.org/id/london-borou...,2014.0,31 days,31
11,2017/1005/TC,Table and Chairs,NaN,NaN,TC,Final Decision,0,2017-03-22,NaN,22/03/2018,...,NaN,NaN,No Spatial Information,11/05/2017 07:36:42 PM,NaN,279,http://opendatacommunities.org/id/london-borou...,2017.0,29 days,29
14,APP\TEN\002545,Temporary Event Notice,Bunga Bunga (Fomerly The Drury Club Basement),"167 DRURY LANE, LONDON, WC2B 5PG",LA2003,Application Approved,No,2008-08-08,NaN,NaN,...,-0.122257,51.515780,Nearest OS Open Map-Local Building,11/05/2017 07:37:25 PM,"(51.51578, -0.122257)",18751,http://opendatacommunities.org/id/london-borou...,2008.0,10 days,10
15,APP\TEN\009839,Temporary Event Notice,Stag,"67 FLEET ROAD, LONDON, NW3 2QU",LA2003,Application Approved,No,2013-11-15,NaN,NaN,...,-0.161684,51.553337,Nearest OS Open Map-Local Building,11/05/2017 07:37:22 PM,"(51.553337, -0.161684)",11873,http://opendatacommunities.org/id/london-borou...,2013.0,1 days,1


In [64]:
# application type, premises name
apptypes = data[:, 1]
#premises = data[:, 2]
wrdcodes = data[:, df.columns.get_loc('Ward Code')]
rcvdates = data[:, df.columns.get_loc('year')]
#apptypes = df["Application Type"]
#premises = df["Premises Name"]

def strnil(s):
    return s if s else ''

labelencoders = {}
onehotencoders = {}

def labels2onehot(col, name):
    enc = preprocessing.OneHotEncoder()
    le = preprocessing.LabelEncoder()
    col = [strnil(c) for c in col]
    le.fit(col)
    #print(list(le.classes_))
    numberencoding = le.transform(col)
    #print('numbers', numberencoding)
    rows = [[n] for n in numberencoding]
    enc.fit(rows)
    onehotencoding = enc.transform(rows).toarray()
    #print(onehotencoding)
    labelencoders[name] = le
    onehotencoders[name] = enc
    return onehotencoding

appenc = labels2onehot(apptypes, 'apptype')
wrdenc = labels2onehot(wrdcodes, 'wrdcode')
#prmenc = labels2onehot(premises)

features = []
for i in range(len(appenc)):
    features += [
        list(appenc[i]) +
        #list(wrdenc[i]) + 
                 [rcvdates[i]]]

print(features[0])
print(len(features[0]))    

N = len(data)
TRAIN = int(N * 0.9)
train_x = features[:TRAIN]
test_x = features[TRAIN:]

train_y = df[:TRAIN]['duration']
test_y = df[TRAIN:]['duration']

#print(N, TRAIN, len(train_y), len(train_x))
#pd.to_timedelta(df.duration).days
#(df.duration.dt / np.timedelta64(1, 'D')).astype(int)

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2005.0]
32


In [65]:
regr = linear_model.LinearRegression()
#regr = linear_model.LogisticRegression()
regr.fit(train_x, train_y)

print('Coefficients: \n', regr.coef_)
print('Mean squared error: %.2f' % np.mean((regr.predict(test_x) - test_y) ** 2))
print('Variance score: %.2f' % regr.score(test_x, test_y))

#plt.scatter(test_x, test_y, color='black')
#plt.plot(test_x, regr.predict(test_x), color='blue', linewidth=3)
#plt.xticks(())
#plt.yticks(())
#plt.show()

Coefficients: 
 [ -7.37699172e+12  -7.37699172e+12  -7.37699172e+12  -7.37699172e+12
  -7.37699172e+12  -7.37699172e+12  -7.37699173e+12  -7.37699172e+12
  -7.37699172e+12  -7.37699173e+12  -7.37699172e+12  -7.37699172e+12
  -7.37699172e+12  -7.37699172e+12  -7.37699173e+12  -7.37699172e+12
  -7.37699172e+12  -7.37699172e+12  -7.37699172e+12  -7.37699173e+12
  -7.37699173e+12  -7.37699172e+12  -7.37699173e+12  -7.37699172e+12
  -7.37699172e+12  -7.37699172e+12  -7.37699172e+12  -7.37699172e+12
  -7.37699172e+12  -7.37699172e+12  -7.37699172e+12  -1.16339844e+02]
Mean squared error: 124501.47
Variance score: -10.72


In [68]:
coefficients = regr.coef_ / np.mean(regr.coef_)

le = labelencoders['apptype']

def coefficient(c):
    [code] = le.transform([c])
    #print(code)
    #print(onehotencoders['apptype'].transform(code).toarray())
    return coefficients[code]


for c in le.classes_:
    print(c, coefficient(c))

CGMP - New / Existing Operator Application 1.03225806451
LPGMP - Conversion Application 1.03225806453
LPGMP - New Application 1.0322580645
Late Temporary Event Notice 1.03225806447
MST Variation 1.03225806451
Massage and Special Treatment New Application 1.0322580645
Massage and Special Treatment Renewal 1.03225806454
Massage and Special Treatment Renewal with variation 1.03225806445
Personal - Change of Address 1.03225806451
Personal - Conversion of Licence 1.03225806462
Personal - New Licence 1.03225806452
Premises - Change of DPS name/address 1.03225806446
Premises - Change of Licensee name/address 1.03225806446
Premises - Change of Premises Name 1.03225806447
Premises - Conversion of Licence 1.03225806461
Premises - Interim Notice 1.03225806453
Premises - Minor Variation of Licence 1.03225806447
Premises - New Licence 1.03225806453
Premises - Provisional Statement 1.03225806451
Premises - Review of Licence 1.03225806458
Premises - Small Variation 1.03225806457
Premises - Variation 